<a href="https://colab.research.google.com/github/GuruPawar-Dev/Chatboat/blob/main/Chatboat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers sentence-transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text


In [ ]:
def split_into_sections(text):
    return text.split("\n\n")  # Splitting by paragraphs


In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

def answer_query(query, context):
    response = qa_pipeline(question=query, context=context)
    return response["answer"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def find_relevant_section(query, sections):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    section_embeddings = embedder.encode(sections, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, section_embeddings)[0]
    best_idx = similarities.argmax()
    return sections[best_idx]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import random

def get_feedback():
    return random.choice(["not helpful", "too vague", "good"])

def adjust_response(query, response, feedback, sections):
    if feedback == "too vague":
        additional_context = find_relevant_section(query, sections)
        return f"{response} (Additional Info: {additional_context[:100]}...)"
    elif feedback == "not helpful":
        return answer_query(query + " Provide more details.", find_relevant_section(query, sections))
    return response


In [ ]:
import logging

logging.basicConfig(filename='support_bot_log.txt', level=logging.INFO)

class SupportBotAgent:
    def __init__(self, document_path):
        self.document_text = extract_text_from_pdf(document_path) if document_path.endswith(".pdf") else extract_text_from_txt(document_path)
        self.sections = split_into_sections(self.document_text)
        logging.info(f"Loaded document: {document_path}")

    def process_query(self, query):
        logging.info(f"Processing query: {query}")

        context = find_relevant_section(query, self.sections)
        if not context:
            return "I don’t have enough information to answer that."

        response = answer_query(query, context)
        print(f"Initial Response: {response}")

        # Feedback loop (max 2 iterations)
        for _ in range(2):
            feedback = get_feedback()
            logging.info(f"Feedback: {feedback}")
            if feedback == "good":
                break
            response = adjust_response(query, response, feedback, self.sections)
            print(f"Updated Response: {response}")

        return response

# Running the bot
if __name__ == "__main__":
    bot = SupportBotAgent("faq.pdf")  # Replace with actual file path
    queries = ["How do I reset my password?", "What’s the refund policy?", "How can I contact support?"]

    for query in queries:
        print(f"Query: {query}")
        print(f"Response: {bot.process_query(query)}\n")


Query: How do I reset my password?
Initial Response: go to the login page
Updated Response: go to the login page and click "Forgot Password
Updated Response: go to the login page and click "Forgot Password
Response: go to the login page and click "Forgot Password

Query: What’s the refund policy?
Initial Response: We offer refunds within 30 days of purchase
Updated Response: We offer refunds within 30 days of purchase
Updated Response: We offer refunds within 30 days of purchase
Response: We offer refunds within 30 days of purchase

Query: How can I contact support?
Initial Response: with your order number to start the process
Updated Response: with your order number to start the process
Updated Response: with your order number to start the process (Additional Info: Resetting Your Password
To reset your password, go to the login page and click "Forgot Password." En...)
Response: with your order number to start the process (Additional Info: Resetting Your Password
To reset your password

In [ ]:
python support_bot_agent.py


SyntaxError: invalid syntax (<ipython-input-18-badebb6f1a0d>, line 1)